## Importing library and creating spark-session

In [47]:
import sys
fileDir = "/home/jovyan/notebooks/"
sys.path.append(fileDir)

from utilities import *
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [48]:
session = create_spark_session("Neo4j GitHub", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


In [49]:
HDFS_URL = "hdfs://namenode:9000//data-team"
PREFIX = "sample_" # or ""

### Reading from HDFS

In [50]:
repositories = session.read.json(f"{HDFS_URL}/{PREFIX}repositories.json") # cambia se ti serve il dataset completo
repositories.printSchema()

root
 |-- repo_name: string (nullable = true)
 |-- watch_count: string (nullable = true)



In [51]:
languages = session.read.json(f"{HDFS_URL}/{PREFIX}languages.json")
languages.printSchema()

root
 |-- language: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bytes: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [52]:
licences = session.read.json(f"{HDFS_URL}/{PREFIX}licences.json")
licences.printSchema()

root
 |-- license: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [53]:
files = session.read.json(f"{HDFS_URL}/{PREFIX}files.json")
files.printSchema()

root
 |-- id: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- path: string (nullable = true)
 |-- ref: string (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- symlink_target: string (nullable = true)



In [54]:
commits = session.read.json(f"{HDFS_URL}/{PREFIX}commits.json") # cambia se ti serve il dataset completo
commits.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- seconds: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- time_sec: string (nullable = true)
 |    |-- tz_offset: string (nullable = true)
 |-- commit: string (nullable = true)
 |-- committer: struct (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- seconds: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- time_sec: string (nullable = true)
 |    |-- tz_offset: string (nullable = true)
 |-- difference: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- new_mode: string (nullable = true)
 |    |    |-- new_path: string (nullable = true)
 |    |    |-- new_repo: string (nullable = true)
 |    |    |-- new_sha1: string (nullable = true)
 |    |    |-- old_mode: string (nullable = true)
 |    |    

## Data Processing

In [32]:
git_commits = commits.select("commit", "subject", "message", "committer.date.seconds", "author.date.seconds")

newColumns = ["id","title","message","committer_date", "author_date"]
git_commits = git_commits.toDF(*newColumns)
git_commits.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- message: string (nullable = true)
 |-- committer_date: string (nullable = true)
 |-- author_date: string (nullable = true)



In [33]:
git_repositories = repositories.withColumnRenamed("repo_name", "name")
git_repositories.printSchema()

root
 |-- name: string (nullable = true)
 |-- watch_count: string (nullable = true)



In [34]:
git_languages = languages.withColumn("name", F.explode(languages["language.name"])).dropDuplicates(["name"]).select("name")
git_languages.printSchema()

root
 |-- name: string (nullable = true)



In [35]:
git_licenses = licences.select("license").withColumnRenamed("license","name").dropDuplicates(["name"])
git_licenses.printSchema()

root
 |-- name: string (nullable = true)



In [36]:
git_files = files.select("id","ref","path","mode","symlink_target")
git_files.printSchema()

root
 |-- id: string (nullable = true)
 |-- ref: string (nullable = true)
 |-- path: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- symlink_target: string (nullable = true)



In [37]:
git_contributor = commits.select("author.*") \
    .union(commits.select("committer.*")) \
    .dropDuplicates(["name"]) \
    .select("name","email")
git_contributor.printSchema()

root
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)



## Writing the nodes in the graph

In [55]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitRepository"

spark_write(SparkConnector.NEO4J, git_repositories, "Append", options=options)

Py4JJavaError: An error occurred while calling o537.save.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Co

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "email"
options["labels"] = ":GitContributor"

spark_write(SparkConnector.NEO4J, git_contributor, "Append", options=options)

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitLanguage"

spark_write(SparkConnector.NEO4J, git_languages, "Append", options=options)

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitLicense"

spark_write(SparkConnector.NEO4J, git_licenses, "Append", options=options)

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "id"
options["labels"] = ":GitFile"

spark_write(SparkConnector.NEO4J, git_files, "Append", options=options)

In [20]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "id"
options["labels"] = ":GitCommit"

spark_write(SparkConnector.NEO4J, git_commits, "Append", options=options)

Dataframe saved to NEO4J


### Writing the relationships in the graph

In [56]:
belongs_to = commits.select("commit","repo")
belongs_to.printSchema()

root
 |-- commit: string (nullable = true)
 |-- repo: string (nullable = true)



In [57]:
stays_in = files.select("id","repo_name") \
    .dropDuplicates(["id","repo_name"])
belongs_to.printSchema()

root
 |-- commit: string (nullable = true)
 |-- repo: string (nullable = true)



In [58]:
parent = commits.select("commit", "parent") \
    .withColumn("parent", F.explode(commits["parent"])) \
    .dropDuplicates(["commit","parent"])
parent.printSchema()

root
 |-- commit: string (nullable = true)
 |-- parent: string (nullable = true)



In [59]:
has = licences.select("repo_name", "license") \
    .dropDuplicates(["repo_name", "license"])
has.printSchema()

root
 |-- repo_name: string (nullable = true)
 |-- license: string (nullable = true)



In [60]:
author = commits.select("author.email", "commit", "author.date.seconds") \
    .withColumnRenamed("seconds","ts")
author = author \
    .filter(author["email"] != "") \
    .withColumn("ts", author["ts"].cast(T.IntegerType())) \
    .dropDuplicates(["email","commit"])
author.printSchema()

root
 |-- email: string (nullable = true)
 |-- commit: string (nullable = true)
 |-- ts: integer (nullable = true)



In [61]:
committed = commits.select("committer.email", "commit", "committer.date.seconds") \
    .withColumnRenamed("seconds","ts") 
committed = committed \
    .filter(committed["email"]!="") \
    .withColumn("ts", committed["ts"].cast(T.IntegerType())) \
    .dropDuplicates(["email","commit"])
committed.printSchema()

root
 |-- email: string (nullable = true)
 |-- commit: string (nullable = true)
 |-- ts: integer (nullable = true)



In [69]:
writted_in = languages.withColumn("lang", F.explode(languages["language"]))
writted_in = writted_in \
    .withColumn("language", writted_in["lang.name"]) \
    .withColumn("bytes", writted_in["lang.bytes"].cast(T.IntegerType())) \
    .select("repo_name", "language", "bytes")

writted_in.printSchema()

root
 |-- repo_name: string (nullable = true)
 |-- language: string (nullable = true)
 |-- bytes: integer (nullable = true)



In [45]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "BELONGS_TO"
options["relationship.save.strategy"] = "keys"

options["relationship.source.labels"] = "GitCommit"
options["relationship.source.node.keys"] = "commit:id"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitRepository"
options["relationship.target.node.keys"] = "repo:name"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, belongs_to, "Append", options=options)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "STAYS_IN"
options["relationship.save.strategy"] = "keys"

options["relationship.source.labels"] = "GitFile"
options["relationship.source.node.keys"] = "id:id"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitRepository"
options["relationship.target.node.keys"] = "repo_name:name"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, stays_in, "Append", options=options)

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "PARENT"
options["relationship.save.strategy"] = "keys"

options["relationship.source.labels"] = "GitCommit"
options["relationship.source.node.keys"] = "commit:id"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitCommit"
options["relationship.target.node.keys"] = "parent:id"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, parent, "Append", options=options)

In [63]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "HAS"
options["relationship.save.strategy"] = "keys"

options["relationship.source.labels"] = "GitRepository"
options["relationship.source.node.keys"] = "repo_name:name"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitLicense"
options["relationship.target.node.keys"] = "license:name"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, has, "Append", options=options)

Dataframe saved to NEO4J


In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "AUTHOR"
options["relationship.save.strategy"] = "keys"
options["reletionship.ts"] = "ts"

options["relationship.source.labels"] = "GitContributor"
options["relationship.source.node.keys"] = "email:email"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitCommit"
options["relationship.target.node.keys"] = "commit:id"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, author, "Append", options=options)

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "COMMITTED"
options["relationship.save.strategy"] = "keys"

options["relationship.source.labels"] = "GitContributor"
options["relationship.source.node.keys"] = "email:email"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitCommit"
options["relationship.target.node.keys"] = "commit:id"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, committed, "Append", options=options)

In [70]:
options = get_default_options(SparkConnector.NEO4J)

options["relationship"] = "WRITTED_IN"
options["relationship.save.strategy"] = "keys"

options["relationship.source.labels"] = "GitRepository"
options["relationship.source.node.keys"] = "repo_name:name"
options["relationship.source.save.mode"] = "Match"

options["relationship.target.labels"] = "GitLanguage"
options["relationship.target.node.keys"] = "language:name"
options["relationship.target.save.mode"] = "Match"

spark_write(SparkConnector.NEO4J, writted_in, "Append", options=options)

Dataframe saved to NEO4J


## Stop spark context and spark session

In [46]:

session.sparkContext.stop()
session.stop()